In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#%cp "-r" "/content/drive/My Drive/Lorenzo/ego-rnn-latest/genAttentionMap.py" "/content/drive/My Drive/Lorenzo/ego-rnn-ss-task/" 

In [0]:
%cd "/content/"
!wget -O "frames.tar.xz" "https://mybucketmldl.s3.amazonaws.com/processed_frames.tar.xz"

/content
--2020-06-03 07:54:46--  https://mybucketmldl.s3.amazonaws.com/processed_frames.tar.xz
Resolving mybucketmldl.s3.amazonaws.com (mybucketmldl.s3.amazonaws.com)... 52.216.229.171
Connecting to mybucketmldl.s3.amazonaws.com (mybucketmldl.s3.amazonaws.com)|52.216.229.171|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579197744 (3.3G) [application/x-xz]
Saving to: ‘frames.tar.xz’

frames.tar.xz       100%[===================>]   3.33G  32.8MB/s    in 1m 45s  

2020-06-03 07:56:32 (32.4 MB/s) - ‘frames.tar.xz’ saved [3579197744/3579197744]



In [0]:
!tar "xf" "frames.tar.xz"

In [0]:
%cd "/content/drive/My Drive/Lorenzo/ego-rnn-latest"

/content/drive/My Drive/Lorenzo/ego-rnn-latest


In [0]:
from __future__ import print_function, division
from objectAttentionModelConvLSTM import *
from spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip)
from makeDatasetFrame import *
import argparse
import sys

DEVICE = "cuda"
VAL_FREQUENCY = 5

In [0]:
def main_run(version, stage, train_data_dir, stage1_dict, out_dir, seqLen, trainBatchSize,
             valBatchSize, numEpochs, lr1, decay_factor, decay_step, mem_size):
    num_classes = 61

    model_folder = os.path.join("./", out_dir, version)

    if os.path.exists(model_folder):
        print('Directory {} exists!'.format(model_folder))
        sys.exit()
    os.makedirs(model_folder)

    train_log_loss = open((model_folder + '/train_log_loss.txt'), 'w')
    train_log_acc = open((model_folder + '/train_log_acc.txt'), 'w')
    val_log_loss = open((model_folder + '/val_log_loss.txt'), 'w')
    val_log_acc = open((model_folder + '/val_log_acc.txt'), 'w')

    # Train val partitioning
    train_usr = ["S1", "S3", "S4"]
    val_usr = ["S2"]

    # Data loader
    normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    spatial_transform = Compose(
        [Scale(256), RandomHorizontalFlip(), MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224),
         ToTensor(), normalize])

    vid_seq_train = makeDataset(train_data_dir, train_usr,
                                spatial_transform=spatial_transform, seqLen=seqLen)

    train_loader = torch.utils.data.DataLoader(vid_seq_train, batch_size=trainBatchSize,
                                               shuffle=True, num_workers=4, pin_memory=True)

    vid_seq_val = makeDataset(train_data_dir, val_usr,
                              spatial_transform=Compose([Scale(256), CenterCrop(224), ToTensor(), normalize]),
                              seqLen=seqLen)

    val_loader = torch.utils.data.DataLoader(vid_seq_val, batch_size=valBatchSize,
                                             shuffle=False, num_workers=2, pin_memory=True, phase="test")

    train_params = []

    # stage 1: train only lstm
    if stage == 1:

        model = attentionModel(num_classes=num_classes, mem_size=mem_size)
        model.train(False)
        for params in model.parameters():
            params.requires_grad = False

    # stage 2: train lstm, layer4, spatial attention and final fc
    else:
        model = attentionModel(num_classes=num_classes, mem_size=mem_size)
        model.load_state_dict(torch.load(stage1_dict))  # pretrained
        model.train(False)
        for params in model.parameters():
            params.requires_grad = False
        #
        for params in model.resNet.layer4[0].conv1.parameters():
            params.requires_grad = True
            train_params += [params]

        for params in model.resNet.layer4[0].conv2.parameters():
            params.requires_grad = True
            train_params += [params]

        for params in model.resNet.layer4[1].conv1.parameters():
            params.requires_grad = True
            train_params += [params]

        for params in model.resNet.layer4[1].conv2.parameters():
            params.requires_grad = True
            train_params += [params]

        for params in model.resNet.layer4[2].conv1.parameters():
            params.requires_grad = True
            train_params += [params]
        #
        for params in model.resNet.layer4[2].conv2.parameters():
            params.requires_grad = True
            train_params += [params]
        #
        for params in model.resNet.fc.parameters():  # fully connected layer
            params.requires_grad = True
            train_params += [params]

        model.resNet.layer4[0].conv1.train(True)
        model.resNet.layer4[0].conv2.train(True)
        model.resNet.layer4[1].conv1.train(True)
        model.resNet.layer4[1].conv2.train(True)
        model.resNet.layer4[2].conv1.train(True)
        model.resNet.layer4[2].conv2.train(True)
        model.resNet.fc.train(True)

    for params in model.lstm_cell.parameters():  # for both stages we train the lstm
        params.requires_grad = True
        train_params += [params]

    for params in model.classifier.parameters():  # for both stages we train the last classifier (after the lstm and avg pooling)
        params.requires_grad = True
        train_params += [params]

    model.lstm_cell.train(True)

    model.classifier.train(True)
    model.cuda()

    loss_fn = nn.CrossEntropyLoss()

    optimizer_fn = torch.optim.Adam(train_params, lr=lr1, weight_decay=4e-5, eps=1e-4)

    optim_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=decay_step,
                                                           gamma=decay_factor)

    train_iter = 0
    min_accuracy = 0

    for epoch in range(numEpochs):
        optim_scheduler.step()
        epoch_loss = 0
        numCorrTrain = 0
        trainSamples = 0
        iterPerEpoch = 0
        model.lstm_cell.train(True)
        model.classifier.train(True)
        if stage == 2:
            model.resNet.layer4[0].conv1.train(True)
            model.resNet.layer4[0].conv2.train(True)
            model.resNet.layer4[1].conv1.train(True)
            model.resNet.layer4[1].conv2.train(True)
            model.resNet.layer4[2].conv1.train(True)
            model.resNet.layer4[2].conv2.train(True)
            model.resNet.fc.train(True)
        for i, (inputs, inputsF, targets) in enumerate(train_loader):
            train_iter += 1
            iterPerEpoch += 1
            optimizer_fn.zero_grad()
            inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).to(DEVICE))
            labelVariable = Variable(targets.to(DEVICE))
            trainSamples += inputs.size(0)
            output_label, _ = model(inputVariable)
            loss = loss_fn(output_label, labelVariable)
            loss.backward()
            optimizer_fn.step()
            _, predicted = torch.max(output_label.data, 1)
            numCorrTrain += (predicted == targets.to(DEVICE)).sum()  # evaluating number of correct classifications
            epoch_loss += loss.data.item()
        avg_loss = epoch_loss / iterPerEpoch
        trainAccuracy = (numCorrTrain.data.item() / trainSamples)

        train_log_loss.write('Training loss after {} epoch = {}\n'.format(epoch + 1, avg_loss))  # log file
        train_log_acc.write('Training accuracy after {} epoch = {}\n'.format(epoch + 1, trainAccuracy))  # log file
        print('Train: Epoch = {} | Loss = {} | Accuracy = {}'.format(epoch + 1, avg_loss, trainAccuracy))

        if (epoch + 1) % VAL_FREQUENCY == 0:
            model.train(False)
            val_loss_epoch = 0
            val_iter = 0
            val_samples = 0
            numCorr = 0
            for j, (inputs, inputsF, targets) in enumerate(val_loader):
                val_iter += 1
                val_samples += inputs.size(0)
                inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).to(DEVICE))
                labelVariable = Variable(targets.to(DEVICE))
                output_label, _ = model(inputVariable)
                val_loss = loss_fn(output_label, labelVariable)
                val_loss_epoch += val_loss.data.item()
                _, predicted = torch.max(output_label.data, 1)
                numCorr += (predicted == targets.to(DEVICE)).sum()  # evaluating number of correct classifications
            val_accuracy = (numCorr.data.item() / val_samples)
            avg_val_loss = val_loss_epoch / val_iter
            print('Val: Epoch = {} | Loss {} | Accuracy = {}'.format(epoch + 1, avg_val_loss, val_accuracy))
            val_log_loss.write('Val Loss after {} epochs = {}\n'.format(epoch + 1, avg_val_loss))  # log file
            val_log_acc.write('Val Accuracy after {} epochs = {}%\n'.format(epoch + 1, val_accuracy))  # log file
            if val_accuracy > min_accuracy:
                save_path_model = (
                        model_folder + '/model_rgb_state_dict.pth')  # every epoch, check if the val accuracy is improved, if so, save that model
                torch.save(model.state_dict(),
                           save_path_model)  # in that way, even if the model overfit, you will get always the best model
                min_accuracy = val_accuracy  # in this way you don't have to care too much about the number of epochs

    train_log_loss.close()
    train_log_acc.close()
    val_log_acc.close()
    val_log_loss.close()

In [0]:
def __main__():
    version = "rgb_16frames_noCAM"
    trainDatasetDir = "/content/"
    outDir = "results"onli
    stage1Dict = "./" + outDir + "/" + version + "_1/model_rgb_state_dict.pth"  # args.stage1Dict

    # STAGE 1 PARAMETERS
    ST1_seqLen = 16  # 7
    ST1_trainBatchSize = 32  # 32
    ST1_valBatchSize = 32  # 32
    ST1_numEpochs = 200  # 200
    ST1_lr1 = 1e-3  # 1e-3
    ST1_stepSize = [25, 75, 150]  # [25, 75, 150]
    ST1_decayRate = 0.1  # 0.1
    ST1_memSize = 512  # 512

    # STAGE 2 PARAMETERS
    ST2_seqLen = 16  # 7
    ST2_trainBatchSize = 32  # 32
    ST2_valBatchSize = 32  # 32
    ST2_numEpochs = 150  # 150
    ST2_lr1 = 1e-4  # 1e-4
    ST2_stepSize = [25, 75]  # [25, 75]
    ST2_decayRate = 0.1  # 0.1
    ST2_memSize = 512  # 512

    # STAGE 1
    
    main_run(version + "_1",
             stage=1,
             train_data_dir=trainDatasetDir,
             stage1_dict=stage1Dict,
             out_dir=outDir,
             seqLen=ST1_seqLen,
             trainBatchSize=ST1_trainBatchSize,
             valBatchSize=ST1_valBatchSize,
             numEpochs=ST1_numEpochs,
             lr1=ST1_lr1,
             decay_factor=ST1_decayRate,
             decay_step=ST1_stepSize,
             mem_size=ST1_memSize)
    
    # STAGE 2
    main_run(version + "_2",
             stage=2,
             train_data_dir=trainDatasetDir,
             stage1_dict=stage1Dict,
             out_dir=outDir,
             seqLen=ST2_seqLen,
             trainBatchSize=ST2_trainBatchSize,
             valBatchSize=ST2_valBatchSize,
             numEpochs=ST2_numEpochs,
             lr1=ST2_lr1,
             decay_factor=ST2_decayRate,
             decay_step=ST2_stepSize,
             mem_size=ST2_memSize)

In [0]:
#!rm "-r" "./results"

In [0]:
__main__()

skipped /content/processed_frames/S1/pour_sugar,spoon,cup/2, different frame number
skipped /content/processed_frames/S1/take_honey/3, different frame number
skipped /content/processed_frames/S1/take_peanut/1, different frame number
skipped /content/processed_frames/S3/pour_coffee,spoon,cup/1, different frame number
skipped /content/processed_frames/S3/pour_coffee,spoon,cup/3, different frame number
skipped /content/processed_frames/S3/pour_sugar,spoon,cup/1, different frame number
skipped /content/processed_frames/S3/pour_sugar,spoon,cup/2, different frame number
skipped /content/processed_frames/S3/pour_sugar,spoon,cup/4, different frame number
skipped /content/processed_frames/S3/stir_spoon,cup/2, different frame number
skipped /content/processed_frames/S4/pour_coffee,spoon,cup/1, different frame number
skipped /content/processed_frames/S4/pour_coffee,spoon,cup/2, different frame number
skipped /content/processed_frames/S4/pour_coffee,spoon,cup/3, different frame number
skipped /con

/content/drive/My Drive/Lorenzo/ego-rnn-latest/MyConvLSTMCell.py:31: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  torch.nn.init.xavier_normal(self.conv_i_xx.weight)
/content/drive/My Drive/Lorenzo/ego-rnn-latest/MyConvLSTMCell.py:32: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  torch.nn.init.constant(self.conv_i_xx.bias, 0)
/content/drive/My Drive/Lorenzo/ego-rnn-latest/MyConvLSTMCell.py:33: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  torch.nn.init.xavier_normal(self.conv_i_hh.weight)
/content/drive/My Drive/Lorenzo/ego-rnn-latest/MyConvLSTMCell.py:35: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  torch.nn.init.xavier_normal(self.conv_f_xx.weight)
/content/drive/My Drive/Lorenzo/ego-rnn-latest/MyConvLSTMCell.py:36: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  torch.nn.init.consta

Train: Epoch = 1 | Loss = 8.471010598269375 | Accuracy = 0.012307692307692308
Train: Epoch = 2 | Loss = 4.829988046125933 | Accuracy = 0.033846153846153845
Train: Epoch = 3 | Loss = 4.095262159000743 | Accuracy = 0.046153846153846156
Train: Epoch = 4 | Loss = 4.08017221364108 | Accuracy = 0.046153846153846156
Train: Epoch = 5 | Loss = 4.00808082927357 | Accuracy = 0.06153846153846154
Val: Epoch = 5 | Loss 3.9054479002952576 | Accuracy = 0.043859649122807015
Train: Epoch = 6 | Loss = 3.978919180956754 | Accuracy = 0.06461538461538462
Train: Epoch = 7 | Loss = 3.9511786590922964 | Accuracy = 0.08615384615384615
Train: Epoch = 8 | Loss = 3.934483203020963 | Accuracy = 0.08
Train: Epoch = 9 | Loss = 3.897888270291415 | Accuracy = 0.08
Train: Epoch = 10 | Loss = 3.8974863182414663 | Accuracy = 0.055384615384615386
Val: Epoch = 10 | Loss 3.7951558232307434 | Accuracy = 0.08771929824561403
Train: Epoch = 11 | Loss = 3.8695308945395728 | Accuracy = 0.052307692307692305
Train: Epoch = 12 | Loss